In [1]:
import keras
from keras import callbacks
from keras.datasets import mnist
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
RN_input = np.load('RN_input.npy')
a = np.load('a.npy')
print (RN_input.shape)
print (a.shape)

(9995, 190, 1536)
(9995,)


In [23]:
RN_input_small = RN_input[0:1000]
a_small = a[0:1000]
print (RN_input_small.shape)
print (a_small.shape)

(1000, 190, 1536)
(1000,)


In [24]:
np.save('RN_input_small', RN_input_small)
np.save('a_small', a_small)

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(a_small)
train_labels = le.transform(a_small)

In [5]:
np.unique(train_labels)
l = len(np.unique(train_labels))

In [6]:
one_hot_labels = keras.utils.to_categorical(train_labels, num_classes=l)
print (one_hot_labels)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [31]:
batch_size = 64
factor = 1
input_shape = RN_input_small.shape[1:3]
g_units = [256,256,256,256]*factor

inputs = Input((input_shape))
g1 = Dense(g_units[0], activation = 'relu')(inputs)
g2 = Dense(g_units[1], activation = 'relu')(g1)
g3 = Dense(g_units[2], activation = 'relu')(g2)
g4 = Dense(g_units[3], activation = 'relu')(g3)
# g_out = Reshape((-1, batch_size, g_units[3]))(g4)
g_sum = Lambda(lambda x: K.sum(x, axis=1))(g4)


f_units = [256*factor,512*factor,l] # for first 10 samples of the RN_input

f1 = Dense(f_units[0], activation = 'relu')(g_sum)
f2 = Dense(f_units[1], activation = 'relu')(f1)
drop = Dropout(0.5)(f2)
f3 = Dense(f_units[2], activation = 'softmax')(drop)  

In [32]:
model = Model(input = inputs, output = f3)

/home/nvaitc/anaconda3/envs/rn/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 190, 1536)         0         
_________________________________________________________________
dense_29 (Dense)             (None, 190, 256)          393472    
_________________________________________________________________
dense_30 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
dense_31 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
dense_32 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
lambda_5 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               65792     
__________

In [34]:
model.compile(optimizer = Adam(lr = 2e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [35]:
# model_checkpoints = callbacks.ModelCheckpoint("weights_{epoch:02d}_{val_loss:.2f}.h5", monitor='val_loss',
#                                                     verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [36]:
model.fit(x = RN_input_small, 
          y = one_hot_labels, 
          epochs=200, 
          validation_split= 0.1, 
          batch_size = batch_size
#           callbacks=[model_checkpoints]
         )

Train on 900 samples, validate on 100 samples
Epoch 1/200
900/900 [==============================] - 2s 2ms/step - loss: 1.9496 - acc: 0.2000 - val_loss: 1.8069 - val_acc: 0.1700
Epoch 2/200
900/900 [==============================] - 1s 1ms/step - loss: 1.8073 - acc: 0.2000 - val_loss: 1.8620 - val_acc: 0.1100
Epoch 3/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7919 - acc: 0.2267 - val_loss: 1.8143 - val_acc: 0.1500
Epoch 4/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7838 - acc: 0.2067 - val_loss: 1.8175 - val_acc: 0.1400
Epoch 5/200
900/900 [==============================] - 1s 2ms/step - loss: 1.7628 - acc: 0.2367 - val_loss: 1.8004 - val_acc: 0.2000
Epoch 6/200
900/900 [==============================] - 1s 2ms/step - loss: 1.7612 - acc: 0.2411 - val_loss: 1.8313 - val_acc: 0.1600
Epoch 7/200
900/900 [==============================] - 1s 1ms/step - loss: 1.7568 - acc: 0.2622 - val_loss: 1.8197 - val_acc: 0.1700
Epoch 8/200
900/900 [==

Epoch 62/200
900/900 [==============================] - 1s 1ms/step - loss: 0.8534 - acc: 0.6767 - val_loss: 2.1594 - val_acc: 0.2900
Epoch 63/200
900/900 [==============================] - 1s 1ms/step - loss: 0.8391 - acc: 0.6833 - val_loss: 2.0118 - val_acc: 0.3300
Epoch 64/200
900/900 [==============================] - 1s 1ms/step - loss: 0.8020 - acc: 0.6911 - val_loss: 2.0973 - val_acc: 0.3400
Epoch 65/200
900/900 [==============================] - 1s 1ms/step - loss: 0.9977 - acc: 0.6333 - val_loss: 2.0749 - val_acc: 0.2600
Epoch 66/200
900/900 [==============================] - 1s 1ms/step - loss: 0.8250 - acc: 0.6967 - val_loss: 2.2651 - val_acc: 0.2200
Epoch 67/200
900/900 [==============================] - 1s 1ms/step - loss: 0.9024 - acc: 0.6611 - val_loss: 1.9229 - val_acc: 0.3000
Epoch 68/200
900/900 [==============================] - 1s 1ms/step - loss: 0.7764 - acc: 0.7067 - val_loss: 2.1137 - val_acc: 0.2800
Epoch 69/200
900/900 [==============================] - 1s 1ms

Epoch 123/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1873 - acc: 0.9500 - val_loss: 3.6262 - val_acc: 0.3100
Epoch 124/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1199 - acc: 0.9678 - val_loss: 3.7439 - val_acc: 0.3700
Epoch 125/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1173 - acc: 0.9700 - val_loss: 4.2037 - val_acc: 0.3000
Epoch 126/200
900/900 [==============================] - 1s 2ms/step - loss: 0.1048 - acc: 0.9711 - val_loss: 4.1850 - val_acc: 0.3200
Epoch 127/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1328 - acc: 0.9700 - val_loss: 4.4624 - val_acc: 0.3400
Epoch 128/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1267 - acc: 0.9733 - val_loss: 4.0284 - val_acc: 0.3500
Epoch 129/200
900/900 [==============================] - 1s 2ms/step - loss: 0.1069 - acc: 0.9733 - val_loss: 4.3768 - val_acc: 0.3100
Epoch 130/200
900/900 [==============================] 

900/900 [==============================] - 1s 1ms/step - loss: 0.1558 - acc: 0.9567 - val_loss: 4.7472 - val_acc: 0.3000
Epoch 184/200
900/900 [==============================] - 1s 2ms/step - loss: 0.1447 - acc: 0.9578 - val_loss: 4.5890 - val_acc: 0.3000
Epoch 185/200
900/900 [==============================] - 1s 1ms/step - loss: 0.1013 - acc: 0.9689 - val_loss: 4.3014 - val_acc: 0.3200
Epoch 186/200
900/900 [==============================] - 1s 1ms/step - loss: 0.0551 - acc: 0.9867 - val_loss: 4.5284 - val_acc: 0.2600
Epoch 187/200
900/900 [==============================] - 1s 1ms/step - loss: 0.0455 - acc: 0.9900 - val_loss: 4.7354 - val_acc: 0.2900
Epoch 188/200
900/900 [==============================] - 1s 1ms/step - loss: 0.0298 - acc: 0.9944 - val_loss: 4.7887 - val_acc: 0.3200
Epoch 189/200
900/900 [==============================] - 1s 2ms/step - loss: 0.0341 - acc: 0.9967 - val_loss: 5.2917 - val_acc: 0.3000
Epoch 190/200
900/900 [==============================] - 1s 1ms/step 